In [36]:
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
# Pull data: Year, player name, position, points, # of Games
col_list = ["Year", "Player", "Pos", "WS"]
df = pd.read_csv("nba_season_stats_kaggle.csv", usecols=col_list)
df.head

<bound method NDFrame.head of          Year             Player  Pos   WS
0      1950.0    Curly Armstrong  G-F  3.5
1      1950.0       Cliff Barker   SG  2.2
2      1950.0      Leo Barnhorst   SF  3.6
3      1950.0         Ed Bartels    F -0.6
4      1950.0         Ed Bartels    F -0.6
...       ...                ...  ...  ...
24686  2017.0        Cody Zeller   PF  5.6
24687  2017.0       Tyler Zeller    C  1.0
24688  2017.0  Stephen Zimmerman    C  0.0
24689  2017.0        Paul Zipser   SF  0.5
24690  2017.0        Ivica Zubac    C  1.1

[24691 rows x 4 columns]>

In [71]:
# Groupby dataset by year and position
# get the average of WS of each position in each year
avg_WS_of_each_pos= df.groupby(["Year","Pos"]).agg({'WS': ['mean']})
WS_Analysis = avg_WS_of_each_pos.reset_index()
WS_Analysis.columns = WS_Analysis.columns.map('|'.join).str.strip('|')
WS_Analysis

,Year,Pos,WS|mean
0,1950.0,C,2.927778
1,1950.0,C-F,2.286667
2,1950.0,F,0.657143
3,1950.0,F-C,1.725000
4,1950.0,F-G,0.858333
...,...,...,...
581,2017.0,PF,1.948739
582,2017.0,PF-C,1.500000
583,2017.0,PG,2.255172
584,2017.0,SF,2.336364


In [72]:
# generate the list of position
position_name = WS_Analysis.iloc[:,1].unique()
positions = position_name.tolist()
positions


['C',
 'C-F',
 'F',
 'F-C',
 'F-G',
 'G',
 'G-F',
 'PF',
 'PG',
 'SF',
 'SG',
 'PF-C',
 'SF-SG',
 'C-PF',
 'SG-SF',
 'PF-SF',
 'SF-PF',
 'SG-PG',
 'SF-PG',
 'C-SF',
 'PG-SG',
 'PG-SF',
 'SG-PF']

In [78]:
# Line Plot
for position in positions:
    WS_Analysis = pd.DataFrame(WS_Analysis.loc[WS_Analysis["Pos"] == positions,:]).reset_index()
    year = WS_Analysis["Year"].to_list()
    WS_mean = WS_Analysis["WS|mean"].to_list()
    WS_Analysis_plt,= plt.plot(year, WS_mean, label= position)

# Cleaning the format    
plt.title('Trend of the WS of each position')
plt.xlabel('Year')
plt.ylabel('Average Win Share') 
plt.legend(bbox_to_anchor=(1.1, 1.5), loc='upper left', borderaxespad=0.)

ValueError: Lengths must match to compare

In [ ]:
# Find the position with maximum PPM of each year